<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao36_EngenhariaPrompt_SelfConsistencyReAct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Engenharia de Prompt Avançada: Self-Consistency e ReAct (Tool Calling) com Gemini

Este notebook demonstra duas técnicas avançadas de engenharia de prompt que vão além do Chain-of-Thought (CoT) básico.

1.  **Self-Consistency:** Como melhorar a confiabilidade das respostas CoT rodando o prompt múltiplas vezes e fazendo uma "votação".
2.  **ReAct (Reason + Act):** Como permitir que o LLM use ferramentas externas para responder perguntas. Mostraremos isso usando a funcionalidade nativa de **Tool Calling** do Gemini, que é a evolução moderna do padrão ReAct.

## Passo 1: Instalação e Configuração

Primeiro, instalamos a biblioteca e configuramos a API Key.

**Instruções (Importante):**
1.  Gere sua API Key no [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  Clique no ícone de **chave** (🔑) na barra lateral esquerda do Colab.
3.  Adicione um novo segredo chamado `GOOGLE_API_KEY` e cole sua chave lá.

In [1]:
!pip install -q google-generativeai

In [2]:
import google.generativeai as genai
from google.colab import userdata
import os
import re
from collections import Counter
import json

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key configurada com sucesso!")
except userdata.SecretNotFoundError:
    print("Erro: 'GOOGLE_API_KEY' não encontrada nos Secrets do Colab.")
    print("Por favor, adicione sua API Key na aba 'Secrets' (ícone de chave) à esquerda.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

API Key configurada com sucesso!


## Parte 2: Self-Consistency (Autoconsistência)

**O Conceito:** O Chain-of-Thought (CoT) é ótimo, mas e se o modelo cometer um erro no "passo a passo"? A resposta final estará errada.

**A Técnica:**
1.  Usamos um prompt CoT (ex: "Pense passo a passo").
2.  Definimos a `temperature` do modelo como > 0 (ex: 0.7). Isso introduz aleatoriedade, fazendo com que o modelo gere *caminhos de raciocínio diferentes* a cada vez.
3.  Rodamos o mesmo prompt N vezes (ex: 5 ou 7 vezes).
4.  Extraímos a resposta final de cada uma das N saídas.
5.  Usamos um "voto majoritário": a resposta final que apareceu mais vezes é provavelmente a correta, pois diferentes caminhos de raciocínio convergiram para ela.

Vamos usar um problema de lógica que pode ter caminhos confusos.

In [3]:
# O problema
problema_sc = """
P: Numa cafeteria, havia 50 muffins. 15 foram vendidos pela manhã.
À tarde, eles assaram mais 30 muffins.
Depois, um cliente comprou 8.
Quantos muffins restam no final do dia?

R: Pense passo a passo.
"""

# 1. Configurar o modelo com temperature > 0 para gerar caminhos diversos
generation_config = {
    "temperature": 0.7,
    "top_p": 1.0,
    "top_k": 32,
}

model_sc = genai.GenerativeModel(
    model_name='gemini-2.5-flash-preview-09-2025',
    generation_config=generation_config
)

# 2. Rodar o prompt N vezes
num_amostras = 5
respostas = []

print(f"Rodando o prompt {num_amostras} vezes (pode levar um momento)...\n")

for i in range(num_amostras):
    try:
        response = model_sc.generate_content(problema_sc)
        respostas.append(response.text)
        print(f"--- Amostra {i+1} ---")
        print(response.text)
        print("--------------------\n")
    except Exception as e:
        print(f"Erro na Amostra {i+1}: {e}")


Rodando o prompt 5 vezes (pode levar um momento)...

--- Amostra 1 ---
Esta é a resolução passo a passo:

**1. Muffins iniciais:**
Havia **50** muffins.

**2. Venda da manhã (Subtração):**
Foram vendidos 15.
$50 - 15 = 35$ muffins restantes.

**3. Assar mais (Adição):**
Eles assaram mais 30.
$35 + 30 = 65$ muffins no total.

**4. Venda final (Subtração):**
Um cliente comprou 8.
$65 - 8 = 57$ muffins restantes.

**Resposta:** Restam **57** muffins no final do dia.
--------------------

--- Amostra 2 ---
Este é o cálculo passo a passo:

1. **Início do Dia:** Havia **50** muffins.
2. **Vendas da Manhã:** 15 foram vendidos.
   $$50 - 15 = 35$$
3. **Assaram Mais:** 30 foram adicionados.
   $$35 + 30 = 65$$
4. **Última Venda:** Um cliente comprou 8.
   $$65 - 8 = 57$$

**Resposta:** Restam **57** muffins no final do dia.
--------------------

--- Amostra 3 ---
Esta é a resolução passo a passo:

**1. Muffins Iniciais:**
Havia **50** muffins.

**2. Venda da Manhã:**
15 foram vendidos.
$50 - 15

In [4]:
# 3. e 4. Extrair a resposta final e fazer a votação

def extrair_resposta_final(texto):
    """Encontra o último número no texto, que geralmente é a resposta final."""
    # Tenta encontrar padrões como "A resposta é 57" ou "restam 57"
    numeros = re.findall(r'\b(\d+)\b', texto)
    if numeros:
        # Retorna o último número encontrado como string
        return numeros[-1]
    return None

respostas_finais_extraidas = []
for r in respostas:
    ans = extrair_resposta_final(r)
    if ans:
        respostas_finais_extraidas.append(ans)

print(f"Respostas finais extraídas: {respostas_finais_extraidas}")

# 5. Voto majoritário
if respostas_finais_extraidas:
    votos = Counter(respostas_finais_extraidas)
    print(f"\Contagem de votos: {votos}")

    resposta_mais_comum, _ = votos.most_common(1)[0]
    print(f"\n🏆 A resposta mais consistente é: {resposta_mais_comum}")
    print("\n(A resposta correta é 50 - 15 + 30 - 8 = 57)")
else:
    print("Nenhuma resposta final pôde ser extraída.")

Respostas finais extraídas: ['57', '57', '57', '57', '57']
\Contagem de votos: Counter({'57': 5})

🏆 A resposta mais consistente é: 57

(A resposta correta é 50 - 15 + 30 - 8 = 57)


<>:23: SyntaxWarning: invalid escape sequence '\C'
<>:23: SyntaxWarning: invalid escape sequence '\C'
/tmp/ipython-input-1655072112.py:23: SyntaxWarning: invalid escape sequence '\C'
  print(f"\Contagem de votos: {votos}")


## Parte 3: ReAct (Reason + Act) via Tool Calling

**O Conceito:** LLMs não sabem de tudo (ex: clima em tempo real, preços de ações, dados em um banco de dados). Eles precisam de "ferramentas". O padrão ReAct (Reason + Act) permite que o modelo:
1.  **Reason (Pensar):** Determine qual informação está faltando.
2.  **Act (Agir):** Decida chamar uma ferramenta (ex: `get_weather(location)`).
3.  **Observe (Observar):** Receba o resultado da ferramenta.
4.  Repita ou gere a resposta final.

A API do Gemini implementa isso nativamente com **Tool Calling**.

**Nosso Exemplo:** Vamos criar uma ferramenta falsa `get_preco_produto` e ver o modelo decidir usá-la.

In [5]:
# 1. Definir nossa ferramenta Python (a função que será chamada)
def get_preco_produto(nome_produto, loja="qualquer"):
    """Retorna o preço de um produto em uma loja específica."""
    print(f"*** Chamada de API Externa: get_preco_produto(nome_produto={nome_produto}, loja={loja}) ***")

    # Simulação de dados de um banco de dados de e-commerce
    db_precos = {
        "laptop gamer": {"preco": 7500.00, "loja": "LojaTech", "estoque": 10},
        "smartphone 5g": {"preco": 3200.50, "loja": "MobileCenter", "estoque": 5},
        "teclado mecânico": {"preco": 450.00, "loja": "PeriféricosTop", "estoque": 20}
    }

    produto_key = nome_produto.lower()
    if produto_key in db_precos:
        info_produto = db_precos[produto_key]
        info_produto["produto"] = nome_produto
    else:
        info_produto = {"produto": nome_produto, "preco": None, "erro": "Produto não encontrado"}

    return json.dumps(info_produto)

# 2. Definir o *esquema* da ferramenta para a API do Gemini
# Isso diz ao modelo O QUE a ferramenta faz e QUAIS são seus argumentos.
ferramenta_preco_produto = {
    "name": "get_preco_produto",
    "description": "Retorna o preço e o estoque de um produto em uma loja online.",
    "parameters": {
        "type": "OBJECT",
        "properties": {
            "nome_produto": {
                "type": "STRING",
                "description": "O nome do produto a ser buscado, ex: Laptop Gamer"
            },
            "loja": {
                "type": "STRING",
                "description": "A loja específica onde buscar (opcional)"
            }
        },
        "required": ["nome_produto"]
    }
}

# 3. Criar o modelo e informar sobre a ferramenta
model_react = genai.GenerativeModel(
    model_name='gemini-2.5-flash-preview-09-2025',
    tools=[ferramenta_preco_produto] # Informa ao modelo quais ferramentas ele pode usar
)

# Iniciar o chat (necessário para o loop de tool calling)
chat = model_react.start_chat(enable_automatic_function_calling=False) # Manual para vermos os passos

print("Modelo ReAct/Tool-Calling configurado.")

Modelo ReAct/Tool-Calling configurado.


In [16]:
# 4. O Loop ReAct (Reason, Act, Observe)


prompt_preco = "Quanto custa o laptop gamer?"
#prompt_preco = "Quanto custa o laptop ?"
#prompt_preco = "Quanto custa o teclado mecânico?"

print(f"Usuário: {prompt_preco}\n")

# PASSO 1: O modelo PENSA (Reason) e decide AGIR (Act)
response_passo1 = chat.send_message(prompt_preco)
candidate = response_passo1.candidates[0]

# Verificar se o modelo decidiu chamar nossa função
if candidate.content.parts[0].function_call:
    # O modelo decidiu AGIR!
    chamada_de_funcao = candidate.content.parts[0].function_call
    print(f"🤖 (Reasoning) O modelo decidiu chamar uma ferramenta: {chamada_de_funcao.name}")
    print(f"🤖 (Action) Argumentos da chamada: {chamada_de_funcao.args}\n")

    # PASSO 2: Nós executamos a ferramenta (a parte "Act" no nosso código)
    # e obtemos a "Observação"
    nome_funcao = chamada_de_funcao.name
    args = chamada_de_funcao.args

    if nome_funcao == "get_preco_produto":
        # Chama a função Python real
        resultado_funcao = get_preco_produto(nome_produto=args['nome_produto'],
                                            loja=args.get('loja', 'qualquer'))

        print(f"🐍 (Observation) Resultado da ferramenta: {resultado_funcao}\n")

        # PASSO 3: Devolver a "Observação" para o modelo
        # O modelo vai usar isso para gerar a resposta final.
        response_passo2 = chat.send_message(content=resultado_funcao)


        # O modelo agora gera a resposta final em linguagem natural
        print("🤖 (Final Answer) Resposta final do modelo:")
        print(response_passo2.text)

    else:
        print(f"Erro: Modelo tentou chamar uma função desconhecida: {nome_funcao}")

else:
    # O modelo não precisou de ferramenta (ou falhou)
    print("🤖 O modelo respondeu diretamente:")
    print(response_passo1.text)

Usuário: Quanto custa o teclado mecânico?

🤖 (Reasoning) O modelo decidiu chamar uma ferramenta: get_preco_produto
🤖 (Action) Argumentos da chamada: <proto.marshal.collections.maps.MapComposite object at 0x7aea9754a1e0>

*** Chamada de API Externa: get_preco_produto(nome_produto=teclado mecânico, loja=qualquer) ***
🐍 (Observation) Resultado da ferramenta: {"preco": 450.0, "loja": "Perif\u00e9ricosTop", "estoque": 20, "produto": "teclado mec\u00e2nico"}

🤖 (Final Answer) Resposta final do modelo:
O teclado mecânico na loja PeriféricosTop custa R$ 450,00 e tem 20 unidades em estoque.


In [8]:
chat.send_message()

google.generativeai.types.content_types.PartDict

## Conclusão

Você viu duas técnicas poderosas:

1.  **Self-Consistency:** Aumenta a **confiabilidade** das respostas de raciocínio complexo através de amostragem e votação. (Usando `temperature` e `Counter`).
2.  **ReAct / Tool Calling:** Dá ao LLM "braços e pernas" para interagir com o mundo exterior, permitindo que ele acesse dados em tempo real e execute ações. (Usando a definição de `tools` e o loop `FunctionCall` / `FunctionResponse`).